# Tennis Match Data Analysis

## How to load and process tennis data?

Please note the following:
- It takes a long time to process the point data. So we avoid processing the point data if possible.
- The raw match data is missing many match_ids. Some match_ids in the point data are not in the original match data. So we need to synchronize them before using the match data.
- Both match data and point data need cleaning.
- We need to process the point data to generate new data columns.

Here are the steps for loading data:
- Fastest: Load the processed point data and processed math data, and then you can start your data analysis.
- Slow: If you download new data from GitHub, you need to clean and process both the raw match data and raw point data.
- Faster: If you have changed some of the match data cleaning and processing functions, you need to load the raw match data and process it. Load the processed point data, and synchronized them. The match data will be updated. The point data is unchanged. Save the processed match data.

## Problem with data

- The original match data has some problem on line 955. Need to manually fix it.
- There are missing values in match data.
- There are duplicate match_ids in the match data.
- Match data has a match_id that is cut off. Need to check the point data to manually fix it.
- There are match_ids in the point data but are not in the match data.
- There are a few cases where a player's handedness is entered wrong.
- Need to manually fix "Final TB?" for Wimbledon 2019 and later.

- There are missing values in point data: "Gm1", "Gm2", "rallyCount". Need to fix them manually.  
- There are rallies without point ending code.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Download the raw match data from https://github.com/JeffSackmann/tennis_MatchChartingProject.
<br>For male players, download charting-m-matches.csv. It's better to convert it to a .xls file.

Some commonly used functions are specified in this section.

## Functions for match data cleaning

In [ ]:
import pandas as pd
import numpy as np
import re
import math

In [ ]:
# Clean match data
def clean_match_data(df_matches):

    df_matches = df_matches.copy()

    # There are several duplicated matches. Remove them.
    df_matches.drop_duplicates(subset="match_id", inplace=True)

    # Data cleaning
    # Remove unnecessary white spaces and convert to lower case letters
    df_matches["Player 1"] = df_matches["Player 1"].str.strip()
    df_matches["Player 2"] = df_matches["Player 2"].str.strip()
    df_matches["Player 1"] = df_matches["Player 1"].replace(to_replace=r"\s+", value="_", regex=True)
    df_matches["Player 2"] = df_matches["Player 2"].replace(to_replace=r"\s+", value="_", regex=True)
    df_matches["Player 1"] = df_matches["Player 1"].str.lower()
    df_matches["Player 2"] = df_matches["Player 2"].str.lower()
    df_matches["Pl 1 hand"] = df_matches["Pl 1 hand"].str.strip()
    df_matches["Pl 2 hand"] = df_matches["Pl 2 hand"].str.strip()
    df_matches["Pl 1 hand"] = df_matches["Pl 1 hand"].str.lower()
    df_matches["Pl 2 hand"] = df_matches["Pl 2 hand"].str.lower()
    df_matches["Tournament"] = df_matches["Tournament"].str.strip()
    df_matches["Tournament"] = df_matches["Tournament"].str.lower()
    df_matches["Tournament"] = df_matches["Tournament"].replace(to_replace=r"\s+", value="_", regex=True)
    df_matches["Surface"] = df_matches["Surface"].str.strip()
    df_matches["Surface"] = df_matches["Surface"].str.lower()
    df_matches["Gender"] = df_matches["Gender"].str.strip()
    df_matches["Gender"] = df_matches["Gender"].str.lower()
    df_matches["Round"] = df_matches["Round"].str.strip()
    df_matches["Round"] = df_matches["Round"].str.lower()

    # Change date to string type
    df_matches = df_matches.astype({"Date": "str"})

    # Fill missing data with "unknown"
    df_matches["Surface"] = df_matches["Surface"].fillna("unknown")
    df_matches["Tournament"] = df_matches["Tournament"].fillna("unknown")
    df_matches["Player 1"] = df_matches["Player 1"].fillna("unknown")
    df_matches["Player 2"] = df_matches["Player 2"].fillna("unknown")
    df_matches["Round"] = df_matches["Round"].fillna("unknown")

    return df_matches

### Tennis match formats

Tennis tournaments use a variety of formats and have a complicated history of format changes. Here is a summary:
- Australian Open: Before 2019, no tiebreak in the 5th set. Since 2019, after 6-6 in the 5th set, there is a 10 point tiebreak.
- Wimbledon: Before 2019, no tiebreak in the 5th set. Since 2019, after 12-12 in the 5th set, there is a 7 point tiebreak.
- Roland Garros: No tiebreak in the 5th set.
- US Open: After 6-6 in the 5th set, there is a 7 point tiebreak.
- NextGen ATP Final: All singles matches are the best-of-five sets, with each set the first to four games (not six games). Tiebreak at 3-All. No-Ad scoring (server’s choice in 2019, receiver's choice in 2018).
- Davis Cup: Before 2016, it's best of 5 sets with no tiebreak in the 5th set (same as Wimbledon and Roland Garros). Since 2016, there is a 7-point tiebreak at 6-6 in the 5th set. Since 2018, all matches are the best of three tiebreak sets.
- Olympics: Before 2016, all the matches are best of three sets, except for the final, which is best of 5 sets, with no tiebreak in the final set of every match. From 2016, all the matches are best of three sets, except for the final, which is best of 5 sets, with a 7-point tiebreak in the final set of every match. From Tokyo 2021, the final match will also be best of 3 sets.
- Laver Cup: Best of 3 sets, with the third set a 10-point tiebreaker.
- All the other tournaments are best of 3 tiebreak sets.

All these can affect the calculation of the game and set score gaps.

The word "tiebreaker" was the original term. "Tiebreak" was used more receently. Both words are used interchangeably.

The function below will set the match format properly.


In [ ]:
# Specify the match format and try to handle all the special cases.
def set_match_format(match_data, match_format_file):

    # This file contains the match format for special cases, such as grand slams.
    match_format = pd.read_csv(match_format_file)

    match_data = match_data.copy()

    # Set the most common match format for professional tennis matches.
    match_data["Best of"] = 3
    match_data["Final TB?"] = 1
    match_data["best_of_game"] = 6
    match_data["regular_tiebreak_trigger"] = 6
    match_data["final_set_tiebreak_trigger"] = 6

    for index, row in match_data.iterrows():
        # We get match information directly from match_id
        match_info = pd.Series(row["match_id"].split("-"), index =
                           ['date','gender', "tournament", "round", "player1", "player2"])

        match_info = match_info.str.lower()

        year = re.findall(r"^(\d{4})", match_info["date"])
        if len(year) > 0:
            match_info["year"] = int(year.pop(0))
        else:
            match_info["year"] = None

        # Handle special cases, such as grand slams, Olympics, Davis Cup, etc.
        for index2, row2 in match_format.iterrows():
            if ((row2["tournament"] in match_info["tournament"]) &
               (match_info["year"] >= row2["year_min"]) &
               (match_info["year"] <= row2["year_max"])):
                if ((row2["round"] == "any") |
                    ((row2["round"] != "any") &
                    (((match_info["round"] == "f") &
                      (row2["round"] == "f")) |
                      ((match_info["round"] != "f") &
                       (row2["round"] != "f"))))):
                    match_data.at[index, "Best of"] = row2["best_of_set"]
                    match_data.at[index, "Final TB?"] = row2["final_tb"]
                    match_data.at[index, "best_of_game"] = row2["best_of_game"]
                    match_data.at[index, "regular_tiebreak_trigger"] = \
                        row2["regular_tiebreak_trigger"]
                    match_data.at[index, "final_set_tiebreak_trigger"] = \
                        row2["final_set_tiebreak_trigger"]
        # print("end of for")

    return match_data

In [ ]:
# Clean the handedness columns because there are errors.
# Create a table of players and their handednesses.
def clean_handedness_match_data(df_matches):

    df_matches = df_matches.copy()

    df_players_handedness = pd.DataFrame()

    players = get_players(df_matches)

    # Check every player and check if there is an error in this player's handedness.
    # Some players may be marked as righthanded in some matches and lefthanded in other matches.
    # A playe cannot have two handedness.
    for player in players:

        #these two lines select the columns "Pl 1 hand" and "Pl 2 hand" from df_matches for each player
        selected_player1_hand = df_matches[(df_matches["Player 1"] == player)]["Pl 1 hand"]
        selected_player2_hand = df_matches[(df_matches["Player 2"] == player)]["Pl 2 hand"]

        #if the player has more than one handedness in Pl 1 hand column,
        #replace the wrong hand with correct hand
        if (len(selected_player_hand_value_counts) > 1):
            print(player)
            print(selected_player_hand_value_counts)

            #finding which hand occurs more frequently for the player
            #this is also the correct handedness for that player
            correct_handedness = selected_player_hand_value_counts.idxmax()

            #finding which hand occurs less frequently for the player
            #this is also the wrong handedness for that player
            wrong_handedness = selected_player_hand_value_counts.idxmin()

            #must save back to the original column and replace wrong_hand with correct_hand
            df_matches.loc[(df_matches["Player 1"] == player), "Pl 1 hand"] = \
                df_matches.loc[(df_matches["Player 1"] == player), "Pl 1 hand"].replace(
                wrong_handedness, correct_handedness)

        #Clean handedness for the "Pl 2 hand" column
        selected_player_hand_value_counts = selected_player2_hand.value_counts()
        if (len(selected_player_hand_value_counts) > 1):
            print(player)
            print(selected_player_hand_value_counts)
            correct_handedness = selected_player_hand_value_counts.idxmax()
            wrong_handedness = selected_player_hand_value_counts.idxmin()
            df_matches.loc[(df_matches["Player 2"] == player), "Pl 2 hand"] = \
                df_matches.loc[(df_matches["Player 2"] == player), "Pl 2 hand"].replace(
                wrong_handedness, correct_handedness)

        row = {"player": player, "handedness": correct_handedness}
        df_players_handedness = df_players_handedness.append(row, ignore_index=True)

    return df_matches, df_players_handedness

## Functions for point data cleaning and pre-processing

In [1]:
import math

# Clean point data
def clean_point_data(df_points):
    df_points = df_points.copy()

    # Someone entered "S" in the "TB?" column to indicate a 10-point super-tiebreak
    # Need to fix it
    df_points["TB?"] = df_points["TB?"].str.replace("S", "2", case=False)
    df_points = df_points.astype({"TB?": "int32"})

    # Set point and set score to numbers
    df_points = df_points.astype({"Pt": "int32"})
    df_points = df_points.astype({"Set1": "int32"})
    df_points = df_points.astype({"Set2": "int32"})

    # Set first and second serve to string type
    df_points = df_points.astype({"1st": "str"})
    df_points = df_points.astype({"2nd": "str"})

    # Check if there are missing values in "Gm1" (player1 game score).
    # If so, we have to manually fix it.
    for index, row in df_points.iterrows():
        if math.isnan(row["Gm1"]):
            print("\"Gm1\" == nan")
            print(row[["match_id", "Pt"]])

    df_points = df_points.astype({"Gm1": "int32"})

    # Check if there are missing values in "Gm2" (player2 game score)
    for index, row in df_points.iterrows():
        if math.isnan(row["Gm2"]):
            print("\"Gm1\" == nan")
            print(row[["match_id", "Pt", "Gm2"]])

    # Set certain columns to the correct data type
    df_points = df_points.astype({"Gm2": "int32"})

    df_points = df_points.astype({"Svr": "int32"})
    df_points = df_points.astype({"Ret": "int32"})

    month_to_number = {"Jan":"1", "Feb":"2", "Mar":"3", "Apr":"4",
                      "May":"5", "Jun":"6", "Jul":"7", "Aug":"8",
                       "Sep":"9", "Oct":"10", "Nov":"11", "Dec":"12"}

    # Convert month strings to numbers
    months = month_to_number.keys()

    for index, row in df_points.iterrows():

        # Check if there are missing values in "rallyCount". We have to manually fix it.
        # Some values in "rallyLength" are incorrect. We have to manually fix it.
        if math.isnan(row["rallyCount"]):
            print("\"rallyCount\" == nan")
            print(row[["match_id", "Pt", "rallyCount"]])

        # Excel automatically changes "2-1" (not 1-2) to "1-Feb", etc. So we have to swap them back.
        # Excel also convert "1-0" to "Jan-00", etc. In this case, do not swap them.
        if any(x in row["Pts"] for x in months):

            # Switch the server-returner score because Excel has changed the sequence.
            score = row["Pts"].split("-")

            if len(score) != 2:
                print("Error: the score is wrong")
                print(row)

            if score[1] in months:
                # If the second score is the name of a month, then switch the score,
                # because Excel will convert "3-2" to "2-Mar", "5-6" to "6-May", etc.
                df_points.at[index, "Pts"] = score[1] + "-" + score[0]

                # If the first score is the name of a month, do nothing,
                # because Excel will convert "1-0" to "Jan-00", "3-0" to "Mar-00", "12-13" to "Dec-13", etc.
                # It doesn't swap the numbers.

    for month in months:
        df_points["Pts"] = df_points["Pts"].str.replace(month, month_to_number[month],
                                                        case=False)

    # Excel will convert "1-0" to "Jan-00" so we need to replace 00 with 0.
    df_points["Pts"] = df_points["Pts"].str.replace("00", "0")

    df_points = df_points.astype({"rallyCount": "int32"})
    df_points = df_points.astype({"PtWinner": "int32"})
    df_points = df_points.astype({"isSvrWinner": "int32"})

    return df_points

## Calculate the winning probabilities for different game scores and point scores.


Calculate winning probabilities for different game score scenarios

In [ ]:
# Calculate winning probability for different game scores
game_score_win_prob = pd.DataFrame(
    columns=['name','serve_first','game_score_player','game_score_opp','win_count','lose_count'])

for index_m, row_m in df_matches.iterrows():
  my_match_id = row_m["match_id"]
  match_format = row_m["Best of"]

  # Get points from one match
  points_in_match, all_sets, all_games = get_points_in_one_match(df_points,
                                                                 my_match_id)

  # Get player names
  player_name1, player_name2 = get_player_names(points_in_match, my_match_id)
  # print(player_name1 + " " + player_name2)

  # Sort the points by the set
  set_indices = np.sort(points_in_match["set_index"].unique())

  for set_index in set_indices:
      points_in_set = points_in_match[points_in_match["set_index"] == set_index]
      points_in_set = points_in_set.sort_values(by="set_index")

      set_winner_id = points_in_set.iloc[-1]["PtWinner"]
      first_server_id = points_in_set.iloc[0]["Svr"]

      # Get the unique game score combinations
      result_df = points_in_set.drop_duplicates(subset=['Gm1', 'Gm2'], keep='first')
      # display(result_df)

      for index, row in result_df.iterrows():
        if set_winner_id == 1:
          win_count = 1
          lose_count = 0
        else:
          win_count = 0
          lose_count = 1

        if first_server_id == 1:
          serve_first = True
        else:
          serve_first = False

        # Add column names for player1
        new_row_player1 = {"name": player_name1, "serve_first": serve_first,
                           "game_score_player": row["Gm1"], "game_score_opp": row["Gm2"],
                           "win_count": win_count, "lose_count": lose_count}

        game_score_win_prob = game_score_win_prob.append(new_row_player1,
                                                      ignore_index = True)

        if set_winner_id == 2:
          win_count = 1
          lose_count = 0
        else:
          win_count = 0
          lose_count = 1

        if first_server_id == 2:
          serve_first = True
        else:
          serve_first = False

        # Add column names for player2
        new_row_player2 = {"name": player_name2, "serve_first": serve_first,
                           "game_score_player": row["Gm2"], "game_score_opp": row["Gm1"],
                           "win_count": win_count, "lose_count": lose_count}

        game_score_win_prob = game_score_win_prob.append(new_row_player2,
                                                      ignore_index = True)

# Calculate the win or lose count for different score scenarios.
# If we want to calculate win or lose count per player, we can add "name" to groupby()
game_score_win_prob = game_score_win_prob.groupby(["serve_first",
                                                   "game_score_player",
                                                   "game_score_opp"]).agg({"win_count":"sum","lose_count":"sum"}).reset_index()

display(game_score_win_prob)

,serve_first,game_score_player,game_score_opp,win_count,lose_count
0,False,0,0,2242,2213
1,False,0,1,1129,1783
2,False,0,2,202,798
3,False,0,3,88,632
4,False,0,4,9,308
...,...,...,...,...,...
121,True,13,13,1,1
122,True,14,13,1,1
123,True,14,14,1,1
124,True,14,15,0,1


In [ ]:
# Calculate win rate
game_score_win_prob["win_rate"] = round((game_score_win_prob["win_count"] / (game_score_win_prob["win_count"] + game_score_win_prob["lose_count"])), 2)

In [ ]:
# Save the data to a csv file
game_score_win_prob.to_csv("/content/gdrive/MyDrive/Tennis_data/w_game_score_win_prob.csv", index=False)

In [ ]:
display(game_score_win_prob)

,serve_first,game_score_player,game_score_opp,win_count,lose_count,win_rate
0,False,0,0,2242,2213,0.50
1,False,0,1,1129,1783,0.39
2,False,0,2,202,798,0.20
3,False,0,3,88,632,0.12
4,False,0,4,9,308,0.03
...,...,...,...,...,...,...
121,True,13,13,1,1,0.50
122,True,14,13,1,1,0.50
123,True,14,14,1,1,0.50
124,True,14,15,0,1,0.00


Calculate the winning percentage for different point score scenarios

In [ ]:
# The game_index is wrong in "20080125-M-Australian_Open-SF-Jo_Wilfried_Tsonga_-Rafael_Nadal"

# Create a dataframe
point_score_win_prob = pd.DataFrame(
    columns=["name","server", "tiebreak",
             "point_score_player", "point_score_opp",
             'win_count','lose_count'])

# This taks too long. Process 1/3 of the matches at a time
for i in range(2200, 3424):
  print("match index: " + str(i))

  # Get "match id" and "best of" from the matches list
  my_match_id = df_matches.iloc[i]["match_id"]
  match_format = df_matches.iloc[i]["Best of"]

  # print(my_match_id)

  # Get points from one match
  points_in_match, all_sets, all_games = get_points_in_one_match(df_points,
                                                            my_match_id)

  # Get player names
  player_name1, player_name2 = get_player_names(points_in_match, my_match_id)
  # print(player_name1 + " " + player_name2)

  # Get points from each set
  for set_index in np.sort(points_in_match["set_index"].unique()):
    points_in_set = points_in_match[(points_in_match["set_index"] == set_index)]
    points_in_set = points_in_set.sort_values(by="Pt") # sort by point index

    # Get points from each game
    for game_index_in_set in np.sort(points_in_set["game_index_in_set"].unique()):
      points_in_game = points_in_set[(points_in_set["game_index_in_set"] == game_index_in_set)]
      points_in_game = points_in_game.sort_values(by="Pt")

      game_winner_id = points_in_game.iloc[-1]["PtWinner"]
      game_server_id = points_in_game.iloc[0]["Svr"]

      # Is this a tiebreak game?
      if points_in_game.iloc[0]["TB?"] == 0:
        is_tiebreak = False
      else:
        is_tiebreak = True

      # Get the unique point scores
      result_df = points_in_game.drop_duplicates(subset=["Pts"], keep='first')
      # display(result_df)

      for index, row in result_df.iterrows():
        # Separate point score because the point score is like 15-40 in the table
        # Need to separate them into 15 and 40
        point_score = row["Pts"].split("-", maxsplit=1)

        # Use player name instead of game_winner_id?
        if game_winner_id == 1:
          win_count = 1
          lose_count = 0
        else:
          win_count = 0
          lose_count = 1

        if game_server_id == 1:
          is_server = True
        else:
          is_server = False

        # If this is a tiebreak game, ignore who serves first by
        # setting is_server to false
        if is_tiebreak:
          is_server = False

        new_row_player1 = {"name": player_name1, "server": is_server,
                          "tiebreak": is_tiebreak,
                          "point_score_player": point_score[0],
                          "point_score_opp": point_score[1],
                          "win_count": win_count, "lose_count": lose_count}

        # Add column names for player1 to the table
        point_score_win_prob = point_score_win_prob.append(new_row_player1,
                                                      ignore_index = True)

        if game_winner_id == 2:
          win_count = 1
          lose_count = 0
        else:
          win_count = 0
          lose_count = 1

        if game_server_id == 2:
          is_server = True
        else:
          is_server = False

        # If this is a tiebreak game, ignore who serves first by
        # setting is_server to false
        if is_tiebreak:
          is_server = False

        # Need to reverse the point score
        new_row_player2 = {"name": player_name2, "server": is_server,
                          "tiebreak": is_tiebreak,
                          "point_score_player": point_score[1],
                          "point_score_opp": point_score[0],
                          "win_count": win_count, "lose_count": lose_count}

        # Add column names for player 2 to the table
        point_score_win_prob = point_score_win_prob.append(new_row_player2,
                                                      ignore_index = True)

# Calculate the win or lose count for different score scenarios grouped, grouped by players.
# If we want to calculate win or lose count per player, we can add "name" to groupby()
point_score_win_prob_with_name = point_score_win_prob.groupby(["name", "server", "tiebreak",
                                                     "point_score_player",
                                                     "point_score_opp"]).agg({"win_count":"sum","lose_count":"sum"}).reset_index()

# Calculate the overall win win or loss count for different score combinations
# Do not add "name" to groupby()
point_score_win_prob_no_name = point_score_win_prob.groupby(["server", "tiebreak",
                                                     "point_score_player",
                                                     "point_score_opp"]).agg({"win_count":"sum","lose_count":"sum"}).reset_index()

# display(point_score_win_prob)

In [ ]:
# Calculate win rate for different score combinations, grouped by players
point_score_win_prob_with_name["win_rate"] = round((point_score_win_prob_with_name["win_count"] / (point_score_win_prob_with_name["win_count"] + point_score_win_prob_with_name["lose_count"])), 2)
point_score_win_prob_with_name.to_csv("/content/gdrive/MyDrive/Tennis_data/m_point_score_win_prob_with_name3.csv", index=False)
display(point_score_win_prob_with_name)

,name,server,tiebreak,point_score_player,point_score_opp,win_count,lose_count,win_rate
0,aaron_krickstein,False,False,0,0,17,78,0.18
1,aaron_krickstein,False,False,0,15,11,28,0.28
2,aaron_krickstein,False,False,0,30,6,7,0.46
3,aaron_krickstein,False,False,0,40,2,0,1.00
4,aaron_krickstein,False,False,15,0,6,50,0.11
...,...,...,...,...,...,...,...,...
12695,younes_el_aynaoui,True,False,40,15,0,1,0.00
12696,younes_el_aynaoui,True,False,40,30,4,2,0.67
12697,younes_el_aynaoui,True,False,40,40,8,1,0.89
12698,younes_el_aynaoui,True,False,40,AD,8,0,1.00


In [ ]:
# Calculate win rate for different score combinations, not grouped by players
point_score_win_prob_no_name["win_rate"] = round((point_score_win_prob_no_name["win_count"] / (point_score_win_prob_no_name["win_count"] + point_score_win_prob_no_name["lose_count"])), 2)
point_score_win_prob_no_name.to_csv("/content/gdrive/MyDrive/Tennis_data/m_point_score_win_prob_no_name3.csv", index=False)
display(point_score_win_prob_no_name)

,server,tiebreak,point_score_player,point_score_opp,win_count,lose_count,win_rate
0,False,False,0,0,7605,28963,0.21
1,False,False,0,15,3842,14692,0.21
2,False,False,0,30,1807,8459,0.18
3,False,False,0,40,913,5340,0.15
4,False,False,15,0,3763,14274,0.21
...,...,...,...,...,...,...,...
118,True,False,40,15,6941,1872,0.79
119,True,False,40,30,6600,2452,0.73
120,True,False,40,40,5961,2284,0.72
121,True,False,40,AD,3796,1484,0.72
